In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os, pdb, re
import keras.preprocessing.text as text
from keras.preprocessing import sequence
from keras import utils
np.set_printoptions(linewidth=300)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/Users/matt/kaggle/movie_sent/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
MAX_WORD_COUNT = 60
N_OUT = 5
BATCH_SIZE = 20
HIDDEN_SIZE = 32
NUM_LAYERS = 2
INIT_SCALE = 0.05

In [3]:
fx = lambda x: pd.read_csv(x,delimiter="\t",header=0)
train_df,test_df = map(fx, ["train.tsv","test.tsv"])
feat_names = train_df.columns.values
X_train, Y_train = train_df['Phrase'].values, train_df['Sentiment'].values
X_test, X_test_ID= test_df['Phrase'].values, test_df['PhraseId'].values
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
test_df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [5]:
Tokenizer = text.Tokenizer()
Tokenizer.fit_on_texts(list(np.concatenate((X_train,X_test),axis=0)))
get_token_key = lambda num: list(Tokenizer.word_index.keys())[list(Tokenizer.word_index.values()).index(num)] #inverse
vocab_size = len(Tokenizer.word_index) + 1
print("There are {0} unique words in data set.".format(vocab_size))

There are 17781 unique words in data set.


In [6]:
def shuffle(a, b): # Shuffles 2 arrays with the same order
    s = np.arange(a.shape[0])
    np.random.shuffle(s)
    return a[s], b[s]
X_train, Y_train = shuffle(X_train,Y_train)


In [7]:
X_train_token, X_test_token = [Tokenizer.texts_to_sequences(x) for x in [X_train,X_test]]

In [8]:
obs_no = 20
print(X_train[obs_no],X_train_token[obs_no],list(map(get_token_key,X_train_token[obs_no])))

is for Angelique [8, 13, 12643] ['is', 'for', 'angelique']


In [9]:
X_train_token_pad, X_test_token_pad = [sequence.pad_sequences(x, maxlen=MAX_WORD_COUNT) for x in [X_train_token,X_test_token]]

In [10]:
X_train_token[20]

[8, 13, 12643]

In [11]:
X_train_token_pad[19:22]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    27,    28,    19,    93,  9435,  2257,  8995],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     8,    13, 12643],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,

In [12]:
Y_train = utils.to_categorical(Y_train, N_OUT)

## Embedding layer - Turns positive integers (indexes) into dense vectors of fixed size”

In [13]:
NUM_STEPS = MAX_WORD_COUNT
x = tf.placeholder(tf.int32,shape=[None,MAX_WORD_COUNT])
y = tf.placeholder(tf.int32,shape=[None,N_OUT])
word_embeddings = tf.Variable(tf.random_uniform([vocab_size, HIDDEN_SIZE])) # NUmber of words by hidden
x_emb = tf.nn.embedding_lookup(word_embeddings, x) #lookup action on the embedding tensor, where each word in the input data set is matched with a row in the embedding tensor, with the matched embedding vector being returned within inputs.